In [2]:
import os
import sys
import subprocess
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
# global variables
base_dir = Path.cwd().parent
seq_dir = base_dir+"seqdata/"
data_dir = base_dir+"data/"
soft_dir = base_dir+"soft/"
outdir = base_dir+"host_pathogen_cross_align_sim/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

r1_fname = seq_dir+"RNA/processed.Kristen/RDW9M8V1_R2_processed.fastq"
r2_fname = seq_dir+"RNA/processed.Kristen/RDW9M8V1_R1_filtered.fastq.gz"

barcodes_fname = seq_dir+"spatial_barcodes.only.txt"

mmul10_fasta_fname = data_dir+"Macaca_mulatta.Mmul_10.dna.toplevel.fa"
mmul10_gff_fname = data_dir+"Macaca_mulatta.Mmul_10.110.gtf"
mmul10_resfeq_gff_fname = data_dir+"Mmul_10_genomic.gtf"

num_threads = 24

In [5]:
%load_ext autoreload
%autoreload 1

sys.path.insert(0, "../scripts/genomic_scripts/")
%aimport definitions

In [12]:
# simlulate expression from SIV239 and test mappability against the macaque genome

# load SIV transcriptome
siv_nt_fa = definitions.load_fasta_dict(outdir+"csess.1.0.0.known.256trimmed.nt.fa",False,True)
print(len(siv_nt_fa))

26


In [16]:
read_len_min = 20
read_len_max = 160
read_len_step = 10

for readlen in range(read_len_min,read_len_max,read_len_step):
    with open(outdir+"reads."+str(readlen)+".fq","w+") as outFP:
        for tx,fa in siv_nt_fa.items():
            for i in range(len(fa) - readlen + 1):
                substr = fa[i:i + readlen]
                outFP.write("@"+tx+"."+str(i)+"."+str(readlen)+"\n")
                outFP.write(substr+"\n")
                outFP.write("+\n")
                outFP.write("F"*len(substr)+"\n")

In [19]:
# now map against the macaque index
for readlen in range(read_len_min,read_len_max,read_len_step):
    reads_fname = outdir+"reads."+str(readlen)+".fq"
    
    cmd = ["/ccb/salz4-4/avaraby/Kristen.ATAC_RNA/soft/STAR/Linux_x86_64_static/STAR",
       "--runThreadN","1",
       "--genomeDir",mmul10_gff_fname.rstrip(".gtf")+".mod.type.STAR",
       "--readFilesIn",outdir,
       "--outFileNamePrefix",outdir+"reads."+str(readlen),
       "--clip3pNbases","0",
       "--clip5pNbases","0",
       "--outFilterType","Normal",
       "--outSAMtype","BAM","SortedByCoordinate",
       "--alignEndsType","Local",
       "--outSAMorder","Paired",
       "--outSAMprimaryFlag","OneBestScore",
       "--outFilterMultimapNmax","20",
       "--outFilterMatchNmin","20",
       "--outSAMmultNmax","1",
       "--outMultimapperOrder","Random",
       "--readMatesLengthsIn","NotEqual",
       "--outFilterMismatchNoverLmax","0.1",
       "--genomeLoad","NoSharedMemory",
       "--limitBAMsortRAM","0",
       "--sjdbGTFfile",mmul10_gff_fname.rstrip(".gtf")+".gffread.gtf",
       "--outSAMunmapped","None"]
    
    print(" ".join(cmd))
    subprocess.call(cmd)

/ccb/salz4-4/avaraby/Kristen.ATAC_RNA/soft/STAR/Linux_x86_64_static/STAR --runThreadN 1 --genomeDir /ccb/salz4-4/avaraby/Kristen.ATAC_RNA/data/Macaca_mulatta.Mmul_10.110.mod.type.STAR --readFilesIn /ccb/salz4-4/avaraby/Kristen.ATAC_RNA/host_pathogen_cross_align_sim/ --outFileNamePrefix /ccb/salz4-4/avaraby/Kristen.ATAC_RNA/host_pathogen_cross_align_sim/reads.20 --clip3pNbases 0 --clip5pNbases 0 --outFilterType Normal --outSAMtype BAM SortedByCoordinate --alignEndsType Local --outSAMorder Paired --outSAMprimaryFlag OneBestScore --outFilterMultimapNmax 20 --outFilterMatchNmin 20 --outSAMmultNmax 1 --outMultimapperOrder Random --readMatesLengthsIn NotEqual --outFilterMismatchNoverLmax 0.1 --genomeLoad NoSharedMemory --limitBAMsortRAM 0 --sjdbGTFfile /ccb/salz4-4/avaraby/Kristen.ATAC_RNA/data/Macaca_mulatta.Mmul_10.110.gffread.gtf --outSAMunmapped None
	/ccb/salz4-4/avaraby/Kristen.ATAC_RNA/soft/STAR/Linux_x86_64_static/STAR --runThreadN 1 --genomeDir /ccb/salz4-4/avaraby/Kristen.ATAC_RNA/

In [15]:
mmul10_gff_fname.rstrip(".gtf")+".gffread.gtf"

'/ccb/salz4-4/avaraby/Kristen.ATAC_RNA/data/Macaca_mulatta.Mmul_10.110.gffread.gtf'